# Парсер вакансий: часть 2

API Telegram: https://core.telegram.org/api \
Для того, чтобы парсить телеграм будем использовать библиотеку [Pyrogram](https://docs.pyrogram.org/intro/quickstart)\
Сперва нам нужно создать приложение, чтобы получить доступ к Telegram API. https://my.telegram.org/apps \
Инструкция по созданию записана, будет лежать в папке вместе с записью и тетрадкой

### Где взять второй номер для экспериментов?

<a href="https://moscow.megafon.ru/corporate/mobile/options/calls/dop_nomer.html">Мегафон</a><br>
<a href="https://msk.tele2.ru/option/second-number">ТЕЛЕ2</a>

Установим pyrogram и tgcrypto (его рекомендуют в документации к Pyrogram):\
https://docs.pyrogram.org/intro/install

In [1]:
!pip install pyrogram tgcrypto

Нам потребуется python-dotenv для автоматизации добавления ключей в окружение чтобы не хранить их в коде

In [2]:
!pip install python-dotenv

Ключи в нашем случае - это **App api_id** и **App api_hash** c сайта telegram https://my.telegram.org/apps

В файл dot.env (находится в папке) нужно вбить свои API_ID и API_HASH

### Импорт библиотек

In [3]:
import pyrogram
import requests
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from pyrogram import Client
import pandas as pd
import json
from datetime import datetime
import re
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets

In [4]:
pyrogram.__version__

'2.0.51'

### Подготовим api_id и api_hash

**ВАЖНО:** Это секретные ключи, поэтому мы их будем брать из environment (файл dot.env в корне нашего проекта)\
Если вы выкладываете код в открытый доступ, проверяйте, чтобы файл *dot.env* не публиковался (например, добавьте в gitignore)

In [5]:
dotenv_path = os.path.join('dot.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [6]:
os.environ['DEMO']

'demo'

In [7]:
API_ID = os.environ['API_ID']
API_HASH = os.environ['API_HASH']

### Как все будет работать

Особенность работы с Telegram API в качестве клиента в том, что нужно проводить авторизацию\
Если бы мы работали в IDE (например, в PyCharm), то мы бы писали сразу весь код там и запускали.\
Однако Jupyter Notebook нагляднее (плюс тут можно удобно показывать результат), поэтому сейчас будет финт ушами.\
Мы запишем код в файлы *.py*. Первый - тестовый для авторизации. Его запустим прямо в терминале, чтобы ввести телефон и код подтверждения.\
Второй - основной для выгрузки сообщений. Мы его можем запустить прямо отсюда.

### Файл авторизации и тестирования `pyro_auth.py` (используем для учебных целей)

```python
#код ниже сохранен в файлике pyro_auth.py здесь он для демонтрации
from pyrogram import Client
import os
from dotenv import load_dotenv


path = os.path.dirname(os.path.abspath(__file__)) #путь

dotenv_path = os.path.join(path + '/dot.env') #прибавляем к пути наш файлик с данными для авторизации
if os.path.exists(dotenv_path): #проверяем существует ли файл
    load_dotenv(dotenv_path) #загружаем файл
    
API_ID = os.environ['API_ID'] #из окружения достаем API ID
API_HASH = os.environ['API_HASH'] #и API HASH
with Client("my_account", API_ID, API_HASH) as app: #задаем название сессии, появится файлик
    app.send_message("me", "Авторизация прошла успешно") #отправим себе сообщение
```

После успешного входа у нас появится файл ```my_account.session```. **Его тоже важно хранить в секрете**

<div class="alert alert-info">
<b>Важно</b>
    
При успешной авторизации у вас приложении (на телефоне или компьютере) появится еще одна дополнительная активная сессия с названием CPython. Увидеть ее можно зайдя в Настройки (settings) -- Устройства (Devices). Вы можете закрыть эту сессию в приложении. Таким образом вы отключите ваш ноутбук (систему) от учетной записи. При необходимости сможете снова авторизироваться в терминале с помощью скрипта `pyro_auth.py`, введя номер телефона и пароль.
    
Также при отказе в будущем от «левого» номера телефона рекомендую удалить свою регистрацию в Телеграм на сайте https://my.telegram.org/ чтобы последующий владелец не видел ваши подписки на каналы и прочее. Также можно удалить регистрацию если вам достался бывший в употреблении номер с действующей регистрацией в Телеграм, и после этого зарегистроваться в Телеграм заново. Так вы избавитесь от мусорных подписок и чужих переписок. <b>Только будьте очень внимательны и не удалите по ошибке свой основной аккаунт!<b/>
    
</div>

### Файл выгрузки (`pyro_run.py`)

Первая часть аналогичная файлу авторизации. Только добавляется pandas
```python
from pyrogram import Client
import os
from dotenv import load_dotenv
import pandas as pd


path = os.path.dirname(os.path.abspath(__file__))

dotenv_path = os.path.join(path + '/dot.env') #
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    
API_ID = os.environ['API_ID']
API_HASH = os.environ['API_HASH']
```

Затем перечислим чаты, из которых мы будем брать сообщения: \
https://t.me/datajob \
https://t.me/foranalysts \
https://t.me/datasciencejobs \
**На выбранные каналы нужно заранее подписаться!**
```python
targets = ['datajob', 'foranalysts', 'datasciencejobs']
```

Дальше создадим список, где будем хранить сообщения:
```python
all_messages = [] 
```

Основная (рабочая) часть нашего кода выгрузки:
```python
try:
    with Client("my_account", API_ID, API_HASH) as app:
        for target in targets:
            for message in app.iter_history(target, limit=20):
                all_messages.append([message.sender_chat, message.message_id, message.date, message.text, message.entities])
    
    df = pd.DataFrame(all_messages)
    df.columns = ["chat", "message_id", "date", "text", "entities"]
    df.to_csv(path + '/telegram.csv', index=False)
    print('Success: ', path + '/telegram.csv')
except Exception as e:
    print('Error: ', e)
```

Запускаем код ниже прямо их тетрадки. Код сохранен в файл (скрипт). Можно запустить скрипт и из теминала (если ранее мы авторизовались и нам не нужно будет вводить номер).

In [10]:
%%bash
python3 parser_script.py #если у вас один питон или Python 3 стоит по умолчанию, то код будет: python pyro_run.py

Traceback (most recent call last):
  File "/mnt/c/Users/dmitr/GitHub/Pet-Projects/Telegram-Parsing/parser_script.py", line 1, in <module>
    from pyrogram import Client
ModuleNotFoundError: No module named 'pyrogram'


CalledProcessError: Command 'b'python3 parser_script.py #\xd0\xb5\xd1\x81\xd0\xbb\xd0\xb8 \xd1\x83 \xd0\xb2\xd0\xb0\xd1\x81 \xd0\xbe\xd0\xb4\xd0\xb8\xd0\xbd \xd0\xbf\xd0\xb8\xd1\x82\xd0\xbe\xd0\xbd \xd0\xb8\xd0\xbb\xd0\xb8 Python 3 \xd1\x81\xd1\x82\xd0\xbe\xd0\xb8\xd1\x82 \xd0\xbf\xd0\xbe \xd1\x83\xd0\xbc\xd0\xbe\xd0\xbb\xd1\x87\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x8e, \xd1\x82\xd0\xbe \xd0\xba\xd0\xbe\xd0\xb4 \xd0\xb1\xd1\x83\xd0\xb4\xd0\xb5\xd1\x82: python pyro_run.py\n'' returned non-zero exit status 1.

### Загрузим полученный файл

In [ ]:
telegram = pd.read_csv('telegram.csv')

In [ ]:
telegram.info()

In [ ]:
telegram = telegram.dropna().reset_index(drop=True) #сбросим индекс

In [ ]:
telegram.head()

#### Посмотрим вакансию

In [ ]:
print(telegram.iloc[3]['text'].lower()) #к нижему регистру чтобы искать регулярные выражения ниже

In [ ]:
telegram

#### Преобразуем дату

In [ ]:
telegram.date = pd.to_datetime(telegram['date'],unit='s')

In [ ]:
telegram.head()

#### Достанем из столбца чат в фомате json IDшник и название чата

In [ ]:
telegram["chat_id"] = telegram["chat"].apply(lambda x: json.loads(x)['id'])
telegram["title"] = telegram["chat"].apply(lambda x: json.loads(x)['title'])

In [ ]:
telegram.head(2)

#### Посмотрим те вакансии где есть, например, аналитик

In [ ]:
telegram[telegram['text'].str.lower().str.contains('analyst')]

In [ ]:
telegram[telegram['chat_id'] == -1001483488834]['text']

<b>Используем регулярные выражения, чтобы получить нужное в сообщениях</b>\
https://docs.python.org/3/library/re.html \
https://regex101.com/

In [ ]:
ds = r'ds|scien|дс|cаен|tensorflow|pytorch'
da = r'analyst|analysis|аналитик|bi|business intelligence|биай|tableau'
de = r'engineer|инженер|spark|airflow'

#### Пометим True или False там где у нас совпадение

In [ ]:
telegram['ds'] = telegram['text'].apply(lambda x: True if re.search(ds, x.lower()) else False)
telegram['da'] = telegram['text'].apply(lambda x: True if re.search(da, x.lower()) else False)
telegram['de'] = telegram['text'].apply(lambda x: True if re.search(de, x.lower()) else False)

In [ ]:
telegram.tail()

In [ ]:
print(telegram.iloc[48]['text'])

In [ ]:
telegram[(telegram['da']) & (telegram['ds']) & (telegram['de'])]

In [ ]:
telegram[telegram['de']]

#### Чтобы нам удобно было смотреть ваканисии добавим кнопку из вчерашнего электива

In [ ]:
button = widgets.Button(description="Показать")
output = widgets.Output()

display(button, output)

counter = 0
def on_clicked(b):
    with output:
        try:
            global counter
            clear_output()
            print('№ вакансии: ', counter)
            print(telegram['text'][counter])
            counter += 1
        except Exception as e:
            print("Вакансии закончились")
            
button.on_click(on_clicked)

### Достанем все ссылки

In [ ]:
def get_links(x, regexp='http\S+'): ###
    try:
        for i in json.loads(x['entities']):
            if i['type'] == 'text_link':
                return i['url']
            elif i['type'] == 'url':
                url = re.findall(regexp, x['text'])[0]
                return url
        
    except:
        return None

In [ ]:
telegram['url'] = telegram.apply(get_links, axis=1)

In [ ]:
telegram['url'][3]

# !

### Парсим ссылки

Документация BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ 
      

In [ ]:
def parse_site(url, element='div', extra_info=None):
    try:
        data = requests.get(url).text ###
        soup = BeautifulSoup(data)
        elements = []
        for extra in extra_info:
            elements += soup.find_all(element, extra)
        html = ''
        for i in elements:
            html += str(i)
        return html
    except Exception as e:
        return '<p>Не удалось получить элемент</p>'

Проверим содержимое

In [ ]:
telegram['url'][14]

In [ ]:
HTML(parse_site(telegram['url'][1], extra_info=[{'class': 'list-parent vacancy-desc'}]))

Нужно перебрать все варианты блоков, где отображена информация о вакансиях.\
Если у нас часто встречаются одни и те же сайты, можно взять доп информацию о блоке и указать ее 

In [ ]:
#заранее добавили какие есть классы на рассматриваемых сайтах

extra_infos = [{'class': 'list-parent vacancy-desc'}, 
               {'class': 'bp-Vacancy__description bp-VacancyDescription'}, 
               {'id': 'vacancy-description'},
               {'class': 'section__block panel js-show'}]

###
telegram['parsed_text'] = telegram['url'].apply(lambda x: parse_site(x, extra_info=extra_infos))

In [ ]:
HTML(telegram['parsed_text'][12])

### Удобный просмотр

In [ ]:
button_2 = widgets.Button(description="Показать вакансии")
output_2 = widgets.Output()

display(button_2, output_2)

counter_2 = 0
def on_clicked_2(b):
    with output_2:
        try:
            global counter_2
            clear_output()
            print('№ вакансии: ', counter_2)
            display(HTML('<h1>Источник: ' + str(telegram['title'][counter_2]) + '</h1>'))
            print(telegram['text'][counter_2])
            display(HTML(telegram['parsed_text'][counter_2]))
            if telegram['url'][counter_2]:
                display(HTML('<a href="' + str(telegram['url'][counter_2]) + '" target="_blank">Ссылка из поста</a>'))
            counter_2 += 1
        except Exception as e:
            print("Вакансии закончились")
            counter_2 = 0
            
button_2.on_click(on_clicked_2)

### Список телеграм каналов, откуда еще можно тащить вакансии (для примера)
https://t.me/data_hr \
https://t.me/biheadhunter \
https://t.me/datajobschannel \
https://t.me/datajobs \
https://t.me/analysts_hunter \
https://t.me/datajob \
https://t.me/analyst_job \
https://t.me/foranalysts \
https://t.me/bds_job \
https://t.me/datajobschannel \
https://t.me/datasciencejobs

### Домашнее задание
Подготовить парсер: hh, telegram и/или другие источники \
Сформировать единую таблицу вакансий из выбранных источников (возможно, в некоторых полях информация будет неполной) \
Сделать визуализацию, построить графики \
Отобрать из данных вакансии для себя


Дедлайн: до 14 апреля 2022

Почта: sekotskiy@yandex-team.ru или в Slack

### Форма обратной связи по элективу "Парсер вакансий"
https://forms.yandex.ru/surveys/11486967.4b4b0013ffddcbd3bb49992226be7e35ec2b66f2/